In [3]:
import torch
import numpy as np
import os

class Evaluator(object):
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class, )*2)
    def Miou(self):
        MIoU = np.diag(self.confusion_matrix)/(np.sum(self.confusion_matrix, axis=1)+np.sum(self.confusion_matrix, axis=0)-
                                              np.diag(self.confusion_matrix))
        MIoU = np.mean(MIoU)
        return MIoU
    def Recall(self):
        recall = np.diag(self.confusion_matrix)/ np.sum(self.confusion_matrix, axis=1)
        Recall = np.mean(recall)
        return Recall
    def Precision(self):
        precision = np.diag(self.confusion_matrix)/ np.sum(self.confusion_matrix, axis=0)
        Precision = np.mean(precision)   
        return Precision
    def generate_matrix(self, gt_image, pre_image):
        mask = (gt_image>=0)&(gt_image<self.num_class)
        label = self.num_class * gt_image[mask].astype('int')+pre_image[mask].astype('int')
        count = np.bincount(label, minlength=self.num_class**2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix
    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape
        self.confusion_matrix += self.generate_matrix(gt_image, pre_image)

In [9]:
from PIL import Image
import os

label_path = './datasets/test/test_labels'   ####测试图ground truth所在路径，确保图像为二值图
pre_path = './datasets/test/test_pred'                           ####测试图测试结果所在路径，确保图像为二值图
all_miou = []
all_recall = []
all_precision = []
labels = os.listdir(label_path)
metric = Evaluator(2)
for label in labels:
    label_name = label.split('.')[0]
    label = Image.open(label_path+'/'+label)
    pre = Image.open(pre_path+'/'+label_name+'.png')
    label = np.array(label).astype(np.float32)
    pre = np.array(pre).astype(np.float32)
    metric.add_batch(label, pre)
    miou = metric.Miou()
    recall = metric.Recall()
    precision = metric.Precision()
    all_miou.append(miou)
    all_recall.append(recall)
    all_precision.append(precision)
iou_sum = sum(all_miou)
recall_sum = sum(all_recall)
precision_sum = sum(all_precision)
iou_ave = np.average(all_miou)
recall_ave = np.average(all_recall)
precision_ave = np.average(all_precision)
print('测试结果平均iou为：%.4f'%iou_ave)
print('测试结果平均recall为：%.4f'%recall_ave)
print('测试结果平均precision为：%.4f'%precision_ave)

测试结果平均iou为：0.8215
测试结果平均recall为：0.8934
测试结果平均precision为：0.8924


In [10]:
all_miou

[0.8373398128628332,
 0.8366169638418464,
 0.8298266792641137,
 0.8316263580675494,
 0.8257135680902592,
 0.8061108946517236,
 0.8096604985346086,
 0.8142138690243241,
 0.8157721317872824,
 0.8171728730093144,
 0.8182027062080605,
 0.8152101133237615]

In [11]:
all_recall

[0.9451226015401932,
 0.9440294504318659,
 0.9078923943252183,
 0.9005315925693558,
 0.8860809907991732,
 0.87951017398222,
 0.8742501116570243,
 0.8733224052488975,
 0.8756021832740069,
 0.8776758372482556,
 0.8796654833359001,
 0.8775263248198066]

In [12]:
all_precision

[0.8687814747980616,
 0.8686735674564536,
 0.8887872782675633,
 0.8984215759361257,
 0.9049339186722318,
 0.882312285561605,
 0.8934593088021144,
 0.9018022489412503,
 0.901478580176067,
 0.9011914747016405,
 0.900459716250705,
 0.8981492396744146]